# Oktoberfest Rescoring API Tutorial

This notebook covers how to run Oktoberfest Rescoring by using API functions. All API functions as well as additional guides and examples can be found on the official read the docs page at https://oktoberfest.readthedocs.io/en/stable/index.html.

The workflow is divided into six sections:

1. Install Oktoberfest and load packages
2. Download example data
3. Preprocess search results and spectra files
4. Annotate spectra and calibrate collision energy
5. Predict intensities and indexed retention time
6. Rescore with percolator
7. Plot the results

# 1. Installation

Before using Oktoberfest, the package and dependencies need to be installed. This step is only required once on the notebook.

## 1.1 
Install percolator.

In [ ]:
!wget https://github.com/percolator/percolator/releases/download/rel-3-06-01/percolator-v3-06-linux-amd64.deb
!dpkg -i percolator-v3-06-linux-amd64.deb

If the install fails due to missing superuser privilages, run this command instead. You will be prompted for your superuser password.

In [ ]:
import getpass
password = str(getpass.getpass()) 
!echo '{password}' | sudo -S bash -c 'dpkg -i percolator-v3-06-linux-amd64.deb'

## 1.2

Install Oktoberfest.

In [18]:
%pip install -q oktoberfest

Note: you may need to restart the kernel to use updated packages.


Import the required packages and functions by executing the code in the cell.

In [19]:
import oktoberfest as ok
from oktoberfest import __version__ as version
import os
import pandas as pd
from pathlib import Path
import urllib.request
import requests
import shutil
from tqdm.auto import tqdm

/cmnfs/home/a.berg/miniconda3/envs/api/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


If this works, you have installed Oktoberfest correctly. You check the version of your installed oktoberfest instance by executing the following. (This notebook was tested with Oktoberfest version 0.7.0)

In [20]:
version

'0.7.0'

# 2. Download example data

The data used in this notebook as an example is provided as a zip archive that can be downloaded from zenodo from this record https://zenodo.org/records/10814834. In case you want to use your own data, skip ahead to Step 3.

Specify the download link as well as file name and download destination.

In [21]:
url = "https://zenodo.org/records/10814834/files/tomato_dataset_example.zip"  # download link
download_dir = Path("./data")                                                 # choose any directory
file_name = Path("tomato_dataset_example.zip")                                # choose any filename

Download and unpack the data. (2.74G, approx. 5 minutes).

In [22]:
if not os.path.isdir(download_dir):
  os.mkdir(download_dir)
download_file = os.path.join(download_dir, file_name)
filesize = int(requests.head(url).headers.get('content-length', -1))
with tqdm(unit="B", total=filesize, unit_scale=True, unit_divisor=1000, miniters=1, desc=url.split("/")[-1]) as t:
    urllib.request.urlretrieve(url=url, filename=download_file, reporthook=lambda blocks, block_size, _: t.update(blocks * block_size - t.n))
shutil.unpack_archive(download_file, download_dir)

tomato_dataset_example.zip: 191MB [00:02, 91.7MB/s]                              


# 3. Pre-Processing

## 3.1 Spectral preprocessing

### A) Converting raw files
If your spectra data is saved in form of .raw files, you need to convert them to .mzml files in order to perform rescoring. In the case that they are already in the mzml data format you can skip to step B). Since Oktoberfest uses the ThermoRawFileParser to convert the files, you need to make sure that you have installed 'mono' and give Oktoberfest the path to the executable.

Install mono if needed. Installation may differ by Linux distribution, refer to https://www.mono-project.com/download/stable/#download-lin.

In [ ]:
password = getpass.getpass()
!echo '{password}' | sudo -S bash -c 'apt install mono-complete -y'

In [ ]:
# path to the ThermoRawFileParser executable
therm = Path("/opt/compomics/ThermoRawFileParser.exe")

Get a list of all raw files located in the specified input directory.

In [ ]:
list_of_spectra = ok.pp.list_spectra(download_dir / file_name.stem,'raw')
print(list_of_spectra)

Convert the raw files to mzml files.

In [ ]:
# Define output directory where individual mzML files will be stored
mzml_directory = download_dir / "mzml/"

# Create the directory if it doesn't exist
if not os.path.exists(mzml_directory):
    os.makedirs(mzml_directory)

# Initialize an empty list to store the paths of the converted mzML files
conversion_results = []

# Loop through the list of spectra files and convert each one
for file in list_of_spectra:
    # Generate corresponding mzML file name for each raw file
    base_filename = os.path.splitext(os.path.basename(file))[0]  # Remove file extension
    output_mzml_file = os.path.join(mzml_directory, f"{base_filename}.mzML")
    
    # Call the convert_spectra function with the current file and the output file path
    ok.pp.convert_raw_to_mzml(file, output_file=output_mzml_file, thermo_exe=therm)
    
    # Append the path of the converted mzML file to the conversion_results list
    conversion_results.append(output_mzml_file)

### B) Loading Spectra
Specify the mzml path and list all mzml files in directory. In case you are using your own data, you need to change the mzml_dir variable to the path of your own mzml directory.

In [23]:
mzml_directory = download_dir / "tomato_dataset_example" # change if needed
mzml_list = ok.pp.list_spectra(mzml_directory,'mzml')
print(mzml_list)

2024-09-06 15:19:36,456 - INFO - oktoberfest.preprocessing.preprocessing::list_spectra Found 1 mzml file in the spectra input directory.
[PosixPath('data/tomato_dataset_example/5407_GC4_063119_S00_U4_R1.mzML')]


If you have multiple spectra files, load them all separately and save them to a list.

In [24]:
spectra_list = []
for spectra_file in mzml_list:
    spectra = ok.pp.load_spectra(filenames=spectra_file, parser="pyteomics")
    spectra_list.append(spectra)
print(spectra_list)

2024-09-06 15:19:36,579 - INFO - spectrum_io.raw.msraw::_read_mzml_pyteomics Reading mzML file: data/tomato_dataset_example/5407_GC4_063119_S00_U4_R1.mzML
[                                                  RAW_FILE  SCAN_NUMBER  \
5407_GC4_063119_S00_U4_R1_3      5407_GC4_063119_S00_U4_R1            3   
5407_GC4_063119_S00_U4_R1_24     5407_GC4_063119_S00_U4_R1           24   
5407_GC4_063119_S00_U4_R1_26     5407_GC4_063119_S00_U4_R1           26   
5407_GC4_063119_S00_U4_R1_27     5407_GC4_063119_S00_U4_R1           27   
5407_GC4_063119_S00_U4_R1_28     5407_GC4_063119_S00_U4_R1           28   
...                                                    ...          ...   
5407_GC4_063119_S00_U4_R1_76832  5407_GC4_063119_S00_U4_R1        76832   
5407_GC4_063119_S00_U4_R1_76833  5407_GC4_063119_S00_U4_R1        76833   
5407_GC4_063119_S00_U4_R1_76834  5407_GC4_063119_S00_U4_R1        76834   
5407_GC4_063119_S00_U4_R1_76835  5407_GC4_063119_S00_U4_R1        76835   
5407_GC4_063119_S00

## 3.2 Loading search results

Converting search results to internal Oktoberfest format. If you are using custom search results, this step isn't necessairy as they should already be provided in the internal data format. Please refer to https://oktoberfest.readthedocs.io/en/stable/internal_format.html for the specific format.

In [25]:
# Define output directory for the converted search results
search_path = Path('./out/msms/msms.prosit')

# Create the directoriy if it doesn't exist
if not os.path.exists(search_path.parent):
    os.makedirs(search_path.parent)

# conveerting maxquant msms.txt to internal format
ok.pp.convert_search(input_path= download_dir / "tomato_dataset_example/msms.txt", output_file=search_path, search_engine='maxquant')

2024-09-06 15:21:03,202 - INFO - spectrum_io.search_result.search_results::generate_internal Found search results in internal format at out/msms/msms.prosit, skipping conversion


,RAW_FILE,SCAN_NUMBER,MODIFIED_SEQUENCE,PROTEINS,PRECURSOR_CHARGE,SCAN_EVENT_NUMBER,MASS,SCORE,REVERSE,SEQUENCE,PEPTIDE_LENGTH
0,5407_GC4_063119_S00_U4_R1,73228,AAAAAGLTVAEVTGSNTVSVAEDELMR,Q5NE18,3,10,2632.30140,214.9200,False,AAAAAGLTVAEVTGSNTVSVAEDELMR,27
1,5407_GC4_063119_S00_U4_R1,41907,AAAAARMNQQK,A0A3Q7H1D4,2,16,1158.59280,9.3831,False,AAAAARMNQQK,11
2,5407_GC4_063119_S00_U4_R1,17706,AAAAFQLEK,A0A3Q7J3B4,2,29,947.50763,27.2540,False,AAAAFQLEK,9
3,5407_GC4_063119_S00_U4_R1,12788,AAAASLLGK,A0A3Q7HMH3,2,9,800.47560,30.5930,False,AAAASLLGK,9
4,5407_GC4_063119_S00_U4_R1,32215,AAAASMVAEDFARR,A0A3Q7EIM9,2,17,1464.71430,17.0260,False,AAAASMVAEDFARR,14
...,...,...,...,...,...,...,...,...,...,...,...
74698,5407_GC4_063119_S00_U4_R1,18859,YYYEPVTGSK,K4DA31,2,12,1205.56050,80.6880,False,YYYEPVTGSK,10
74699,5407_GC4_063119_S00_U4_R1,55236,YYYKFPSPTK,A0A3Q7EMA7,2,10,1292.64410,11.8480,False,YYYKFPSPTK,10
74700,5407_GC4_063119_S00_U4_R1,73593,YYYKSPTPSKYYYK,A0A3Q7F714,2,4,1849.89270,11.8660,False,YYYKSPTPSKYYYK,14
74701,5407_GC4_063119_S00_U4_R1,73963,YYYNSQTNVSQWEHPSK,A0A3Q7IPQ3,2,6,2129.94430,5.4839,False,YYYNSQTNVSQWEHPSK,17


Loading search results to a Pandas dataframe.

In [26]:
peptide_df = ok.pp.load_search(search_path)
peptide_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74703 entries, 0 to 74702
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RAW_FILE           74703 non-null  object 
 1   SCAN_NUMBER        74703 non-null  int64  
 2   MODIFIED_SEQUENCE  74703 non-null  object 
 3   PROTEINS           74703 non-null  object 
 4   PRECURSOR_CHARGE   74703 non-null  int64  
 5   SCAN_EVENT_NUMBER  74703 non-null  int64  
 6   MASS               74703 non-null  float64
 7   SCORE              74703 non-null  float64
 8   REVERSE            74703 non-null  bool   
 9   SEQUENCE           74703 non-null  object 
 10  PEPTIDE_LENGTH     74703 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 5.8+ MB


Filtering search results using given peptide length and charge requirements posed by the selected model.

In [27]:
filtered_peptides = ok.pp.filter_peptides_for_model(peptides=peptide_df, model='prosit')

# 4. Annotate spectra and calibrate collision energy

Annotation and collision energy calibration is done seperately for each raw file so you need to loop over all spectra files in the spectra_list. 
The spectra files need to be merged on "Raw_file" and "Scan number" with the peptides taken from the search results before annotation.

In [28]:
psm_list = []
for spectra in spectra_list:
    psms = ok.pp.merge_spectra_and_peptides(spectra=spectra, search=filtered_peptides)
    psm_list.append(psms)

2024-09-06 15:21:03,832 - INFO - oktoberfest.preprocessing.preprocessing::merge_spectra_and_peptides Merging rawfile and search result


Annotate the peaks for all psms in each psm_file (by efault b- and y-ions). It is important to check the mass_tolerance of the data used in order to correctly set the mass_tol and unit_mass_toll parameters. More information can be found in the docs https://oktoberfest.readthedocs.io/en/stable/config.html.

In [29]:
library_list = []
for psms in psm_list:
    library = ok.pp.annotate_spectral_library(psms=psms, mass_tol=20, unit_mass_tol='ppm')
    library_list.append(library)

2024-09-06 15:21:03,946 - INFO - oktoberfest.preprocessing.preprocessing::annotate_spectral_library Annotating spectra...


/cmnfs/home/a.berg/miniconda3/envs/api/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2024-09-06 15:25:37,804 - INFO - oktoberfest.preprocessing.preprocessing::annotate_spectral_library Finished annotating.


Perform ce_calibration on each annotated library with the intended model and ce range.

In [30]:
alignment_list = []
for library in library_list:
    alignment_library = ok.pr.ce_calibration(library, ce_range=(19,50),group_by_charge=False,model_name="Prosit_2020_intensity_HCD",  
                  server_url="koina.wilhelmlab.org:443")

Getting predictions: 100%|██████████| 31/31 [00:07<00:00,  3.95it/s]


Determine and assign the best collision energy.

In [31]:
for library in library_list:
    ce_alignment = alignment_library.obs.groupby(by=["COLLISION_ENERGY"])["SPECTRAL_ANGLE"].mean()
    best_ce = ce_alignment.idxmax()
    library.obs['COLLISION_ENERGY'] = best_ce

# 5. Predict intensities and indexed retention time

In [32]:
for library in library_list:
    ok.pr.predict_rt(data = library, server_url="koina.wilhelmlab.org:443", model_name="Prosit_2019_irt", ssl=True)
    ok.pr.predict_intensities(data=library, server_url="koina.wilhelmlab.org:443", model_name="Prosit_2020_intensity_HCD", ssl=True)

Getting predictions: 100%|██████████| 75/75 [00:16<00:00,  4.45it/s]


# 6. Rescore with percolator

Create output dictionary for percolator results.

In [33]:
percolator_dir = Path('./out/percolator')

if not os.path.exists(percolator_dir):
    os.makedirs(percolator_dir)

Create two feature files for each spectra files that will serve as input for percolator. The original file containing only original features from the search results as well as the rescore file containing additional features calculated by Oktoberfest.

In [34]:
# lists to save feature files in order to merge them into one later
rescore_files = []
original_files = []

for library, spectra_file in zip(library_list,mzml_list):
    # create output paths for feature files
    original_features = percolator_dir / spectra_file.with_suffix(".original.tab").name
    original_files.append(original_features)
    rescore_features = percolator_dir / spectra_file.with_suffix(".rescore.tab").name
    rescore_files.append(rescore_features)

    # generate features for original file 
    ok.re.generate_features(library=library, search_type='original', output_file=original_features, all_features=False, regression_method='spline')
    # generate features for rescore file 
    ok.re.generate_features(library=library, search_type='rescore', output_file=rescore_features, all_features=False, regression_method='spline')

/cmnfs/home/a.berg/miniconda3/envs/api/lib/python3.10/site-packages/spectrum_fundamentals/metrics/similarity.py:239: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  scipy.stats.pearsonr(obs, pred)[0] if method == "pearson" else scipy.stats.spearmanr(obs, pred)[0]


2024-09-06 15:28:40,725 - INFO - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Found 33178 (out of 55207) targets below 0.01             FDR using spectral_angle as feature
2024-09-06 15:28:40,730 - INFO - spectrum_fundamentals.metrics.percolator::apply_lda_and_get_indices_below_fdr Found 33178 targets and 19496 decoys as input for the LDA model
2024-09-06 15:28:41,557 - INFO - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Found 35579 (out of 55207) targets below 0.01             FDR using lda_scores as feature
2024-09-06 15:28:41,693 - INFO - spectrum_fundamentals.metrics.percolator::calc Median absolute error predicted vs observed retention time on targets < 1% FDR: 1.7724386179814395


Merge all feature files into one combined original and one combined rescore file.

In [35]:
merged_rescore = percolator_dir / "rescore.tab"
ok.re.merge_input(rescore_files, merged_rescore)

merged_original = percolator_dir / "original.tab"
ok.re.merge_input(original_files, merged_original)

Rescore with percolator.

In [36]:
!module av

------------------------ /usr/share/modules/modulefiles ------------------------
dot  module-git  module-info  modules  null  use.own  

----------------------- /cmnfs/data/cluster/modulefiles ------------------------
dotnet/3.1.426  fragpipe/22.0      maxquant/2.4.4.0  percolator/3.6.5           
dotnet/8.0.302  maxquant/1.6.1.0   maxquant/2.4.7.0  sage/0.14.7                
ENB/search_v1   maxquant/1.6.2.10  maxquant/2.6.3.0  thermorawfileparser/1.4.2  
ENB/search_v2   maxquant/2.4.2.0   percolator/3.6.1  thermorawfileparser/1.4.3  

Key:
modulepath  
>

In [37]:
!module load percolator/3.6.5

In [38]:
!module use percolator/3.6.5

In [ ]:
ok.re.rescore_with_percolator(merged_rescore, percolator_dir)
ok.re.rescore_with_percolator(merged_original, percolator_dir)

# 7. Plot the results

In [ ]:
# Plots path 
plots_dir = Path('./out/plots')

if not os.path.exists(percolator_dir):
    os.makedirs(percolator_dir)

# Generate histogram of the score distribution for targets and decoys.
peptide_decoy_df = pd.read_csv(percolator_dir / 'rescore.percolator.decoy.peptides.txt', sep='\t')

peptide_target_df = pd.read_csv(percolator_dir / 'rescore.percolator.peptides.txt', sep='\t')

display(peptide_decoy_df)

ok.pl.plot_score_distribution(peptide_target_df, peptide_decoy_df, level='peptide', filename=plots_dir / "score_distribution")

In [ ]:
ok.pl.plot_all(plots_dir)